# European Develompent Indicators - Iceland

## Research Questions

1. How has Iceland's GDP growth correlated with inflation rates over the past 20 years?

2. What is the relationship between educational attainment and unemployment in Iceland?

3. What is the gender wage gap trend in Iceland, and how does it compare with female employment rates?

## Datasets

* [namq_10_gdp](https://doi.org/10.2908/NAMQ_10_GDP) - Eurostat: Quarterly GDP

* [prc_hicp_midx](https://doi.org/10.2908/PRC_HICP_MIDX) - Eurostat: Monthly Harmonized CPI

* [prc_hicp_manr](https://doi.org/10.2908/PRC_HICP_MANR) - Eurostat: Monthly CPI inflation rate

* [lfsa_urgaed](https://doi.org/10.2908/LFSA_URGAED) - Eurostat: Quarterly unemployment rate by education level

* [lfsa_egaed](https://doi.org/10.2908/LFSA_EGAED) - Eurostat: Quarterly employment rate by education level

* [une_rt_m](https://doi.org/10.2908/UNE_RT_M) - Eurostat: Monthly total unemployment rate

* [edat_lfse_03](https://doi.org/10.2908/EDAT_LFSE_03) - Eurostat: Annual population by educational attainment level

* [nama_10_co3_p3](https://doi.org/10.2908/NAMA_10_CO3_P3) - Eurostat: Annual household disposable income

* [earn_nt_net](https://doi.org/10.2908/EARN_NT_NET) - Eurostat: Annual net earnings


## Imports

In [ ]:
import os
import eurostat
import pandas as pd

## Parameters

In [ ]:
COUNTRY = {
    "name": "Iceland",
    "code": "IS"
}

DATAFRAMES = {
    "quarterly_gdp": "namq_10_gdp",
    "monthly_hcpi": "prc_hicp_midx",
    "monthly_cpi_inflation": "prc_hicp_manr",
    "quarterly_unemployment_education": "lfsa_urgaed",
    "quarterly_employment_education": "lfsa_egaed",
    "monthly_unemployment": "une_rt_m",
    "annual_education_attainment": "edat_lfse_03",
    "annual_household_income": "nama_10_co3_p3",
    "annual_net_earnings": "earn_nt_net",
}

DATA_PATH = "data/"

In [ ]:
# Ensure data directory exists
# os.makedirs(DATA_PATH, exist_ok=True)

# Fetch and save each dataframe
# for name, code in DATAFRAMES.items():
# name, code = list(DATAFRAMES.items())[0]
    # if os.path.exists(os.path.join(DATA_PATH, f"{name}.csv")):
    #     print(f"File for {name} already exists. Skipping download.")
    #     df = pd.read_csv(os.path.join(DATA_PATH, f"{name}.csv"))
    #     print(df.head())
    #     continue

    # try:
# df = eurostat.get_data(code)
# print(df)
print("")
# print(df.describe())
# print(df.head())




In [ ]:
for name in DATAFRAMES.keys():
    file_path = os.path.join(DATA_PATH, f"{name}.tsv", sep='\t')
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"{name}: {len(df)} rows, columns: {list(df.columns)}")
    else:
        print(f"File {file_path} does not exist.")